<a href="https://colab.research.google.com/github/crunchdomo/llm_conversation/blob/main/test_base_langchain_openai_baselne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [25]:
!pip install langchain_openai
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.5 MB/s eta 0:00:00


In [18]:
from google.colab import userdata
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import tool # Import the tool decorator
from pydantic import BaseModel # Import BaseModel

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Initialize chef LLM
chef_llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.7)

# Define the helper functions (these are NOT the tools directly)
def parse_steps(recipe_text):
    # Simple example: split by sentences or numbered steps
    return [step.strip() for step in recipe_text.split('.') if step.strip()]

def extract_ingredients(recipe_text):
    # Placeholder: you might use regex or an LLM call to extract ingredients
    # For now, just return a dummy list
    return ["ingredient1", "ingredient2"]

# Define agent tools using the @tool decorator and BaseModel
class RecipeInput(BaseModel):
    recipe_text: str

@tool(args_schema=RecipeInput)
def process_recipe(recipe_text: str) -> dict:
    """Extract structured recipe data from user input"""
    return {
        "steps": parse_steps(recipe_text),
        "ingredients": extract_ingredients(recipe_text)
    }

@tool
def explain_cooking_technique(technique: str) -> str:
    """Explain cooking methods and safety tips"""
    return f"Detailed explanation of {technique}..."

# Use the correctly defined tools list
tools = [process_recipe, explain_cooking_technique]


# Create agent prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a master chef assistant. Guide users through recipes using tools."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Build agent
agent = create_openai_tools_agent(chef_llm, tools, prompt)

# Initialize memory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,  # Maintain last 5 exchanges
    return_messages=True
)

# Pass the memory object to the AgentExecutor
# Use the corrected tools list
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

<ipython-input-18-5df565167d14>:58: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [19]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,  # Maintain last 5 exchanges
    return_messages=True
)


In [20]:
from langchain.tools import tool
from pydantic import BaseModel

class RecipeInput(BaseModel):
    recipe_text: str

@tool(args_schema=RecipeInput)
def process_recipe(recipe_text: str) -> dict:
    """Extract structured recipe data from user input"""
    return {
        "steps": parse_steps(recipe_text),
        "ingredients": extract_ingredients(recipe_text)
    }

@tool
def explain_cooking_technique(technique: str) -> str:
    """Explain cooking methods and safety tips"""
    return f"Detailed explanation of {technique}..."

tools = [process_recipe, explain_cooking_technique]


In [21]:
from langchain.chains import LLMChain

user_llm = ChatOpenAI(model="gpt-3.5-turbo")

user_prompt = ChatPromptTemplate.from_template(
    "Process this recipe input: {input}"
)

user_chain = LLMChain(llm=user_llm, prompt=user_prompt)


<ipython-input-21-ea6629330e45>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  user_chain = LLMChain(llm=user_llm, prompt=user_prompt)


In [27]:
# Get recipe input from user
recipe_input = input("Paste your recipe: ")

# Process with user_chain (optional, depending on your workflow)
processed_recipe = user_chain.invoke({"input": recipe_input})

# Start the chef agent session
response = agent_executor.invoke({"input": recipe_input})
print(response['output'])


KeyboardInterrupt: Interrupted by user

In [28]:
from langchain_core.messages import HumanMessage, AIMessage

def interactive_cooking_session():
    # Initialize conversation
    print("Chef: Welcome! Let's cook something amazing. What recipe are we making today?")

    while True:
        # Get user input
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", "done"]:
            print("Chef: Happy cooking! Let me know if you need more help.")
            break

        # Process input through agent
        response = agent_executor.invoke(
            {"input": user_input},
            {"callbacks": [memory]}
        )

        # Display agent response
        print(f"Chef: {response['output']}")


In [29]:
from langgraph.graph import StateGraph

class CookingState(BaseModel):
    current_step: int = 0
    ingredients_confirmed: bool = False
    tools_ready: bool = False
    recipe_loaded: bool = False


# Define agent nodes
builder = StateGraph(CookingState)

def chef_node(state):
    response = agent_executor.invoke(state)
    return {"messages": [AIMessage(content=response["output"])]}

def sous_chef_node(state):
    # Different LLM configuration
    sous_chef_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)
    response = sous_chef_llm.invoke(state["messages"])
    return {"messages": [AIMessage(content=response.content)]}

builder.add_node("chef", chef_node)
builder.add_node("sous_chef", sous_chef_node)
builder.add_edge("chef", "sous_chef")
builder.add_edge("sous_chef", "chef")


In [ ]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", """You're a master chef assistant. Guide users through recipes using tools.
# Current State:
# - Step: {current_step}
# - Ingredients Confirmed: {ingredients_confirmed}
# - Tools Ready: {tools_ready}"""),
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("human", "{input}"),
#     MessagesPlaceholder(variable_name="agent_scratchpad")
# ])


In [30]:
#!pip install langchain-openai python-dotenv

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import tool
from pydantic import BaseModel
import os

# Initialize LLM and memory
chef_llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.7)
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True
)

# Helper functions
def parse_steps(recipe_text):
    return [step.strip() for step in recipe_text.split('\n') if step.strip()]

def extract_ingredients(recipe_text):
    return [line.strip() for line in recipe_text.split('\n') if "tsp" in line or "cup" in line]

# Tool definitions
class RecipeInput(BaseModel):
    recipe_text: str

@tool(args_schema=RecipeInput)
def process_recipe(recipe_text: str) -> dict:
    """Extract structured recipe data from user input"""
    return {
        "steps": parse_steps(recipe_text),
        "ingredients": extract_ingredients(recipe_text)
    }

@tool
def explain_cooking_technique(technique: str) -> str:
    """Explain cooking methods and safety tips"""
    return f"Detailed explanation of {technique}..."

tools = [process_recipe, explain_cooking_technique]

# Agent setup
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a master chef assistant. Guide users step-by-step through the recipe. Present one instruction at a time and wait for the user to say 'next' before continuing."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


agent = create_openai_tools_agent(chef_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

# Conversation interface
def cooking_session():
    print("Chef: Welcome to your virtual cooking assistant! What recipe are we preparing today?")

    while True:
        user_input = input("\nYou: ")

        if user_input.lower() in ["exit", "quit", "done"]:
            print("\nChef: Bon appétit! Let me know if you need further assistance.")
            break

        response = agent_executor.invoke({"input": user_input})
        print(f"\nChef: {response['output']}")

# Start session
if __name__ == "__main__":
    cooking_session()


Chef: Welcome to your virtual cooking assistant! What recipe are we preparing today?


KeyboardInterrupt: Interrupted by user

In [35]:
# Add trainee agent class
import re
class TraineeAgent:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
        self.current_step = 0
        self.recipe_data = None

    def process_initial_recipe(self, recipe_text):
        """Parse recipe structure with numbered step handling"""
        # Extract numbered steps first
        steps = re.split(r'\n\s*\d+\.\s', recipe_text)
        if len(steps) > 1:
            structured_recipe = {
                'steps': [s.strip() for s in steps[1:]]  # Skip first empty split
            }
            self.recipe_data = structured_recipe
            return self.recipe_data
        else:
            # Fallback to original processing
            self.recipe_data = process_recipe.invoke({"recipe_text": recipe_text})
            return self.recipe_data


    def generate_response(self, chef_message):
        """Generate trainee's response to chef's instruction"""
        response = self.llm.invoke([HumanMessage(
            content=f"""You're a cooking trainee following instructions. Current step: {self.current_step+1}/{len(self.recipe_data['steps'])}.
            Last chef instruction: {chef_message}
            Respond with either:
            1. "next" to continue
            2. A clarification question about technique/ingredients
            3. "repeat" to hear the step again"""
        )])
        return response.content.strip().lower()

# Modified conversation interface
def automated_cooking_session(recipe_json):
    # Initialize agents
    trainee = TraineeAgent()
    recipe_data = trainee.process_initial_recipe(recipe_json)

    print("Chef: Let's begin cooking Miso-Butter Roast Chicken!")

    for step_idx, step in enumerate(recipe_data['steps']):
        # Chef provides instruction
        chef_response = agent_executor.invoke({
            "input": f"Current step {step_idx+1}: {step}"
        })
        print(f"\nChef: {chef_response['output']}")

        # Trainee interaction loop
        while True:
            trainee_response = trainee.generate_response(chef_response['output'])

            if "next" in trainee_response:
                trainee.current_step = step_idx
                break

            elif "repeat" in trainee_response:
                print(f"Trainee: [Requesting repeat]")
                continue  # Re-use same step content

            else:  # Handle questions
                clarification = agent_executor.invoke({
                    "input": f"Trainee asks: {trainee_response}"
                })
                print(f"Trainee: {trainee_response}")
                print(f"Chef: {clarification['output']}")

    print("\nChef: All steps completed! Enjoy your meal!")

# Example usage
if __name__ == "__main__":
    recipe_json = '''{
            "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
            "ingredients": [
                '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
                '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
                '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
                '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
                '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
                '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
                '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
                '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
                '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
                '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
            ],
            "instructions": """
                1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

                2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

                3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

                4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
                Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

                5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

                6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

                7. Using your fingers, mash flour and butter in a small bowl to combine.

                8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

                9. Serve chicken with gravy and squash panzanella alongside.
                    """
        }
    }'''

    automated_cooking_session(recipe_json)


Chef: Let's begin cooking Miso-Butter Roast Chicken!


> Entering new AgentExecutor chain...
Step 1: Pat the chicken dry with paper towels, then season it all over with 2 teaspoons of salt. Tie the legs together using kitchen twine to help it cook evenly. Let the chicken sit at room temperature for 1 hour to allow the seasoning to penetrate and the chicken to come to a more even cooking temperature.

> Finished chain.

Chef: Step 1: Pat the chicken dry with paper towels, then season it all over with 2 teaspoons of salt. Tie the legs together using kitchen twine to help it cook evenly. Let the chicken sit at room temperature for 1 hour to allow the seasoning to penetrate and the chicken to come to a more even cooking temperature.


> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [36]:
# Add trainee agent class
class TraineeAgent:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
        self.current_step = 0
        self.recipe_data = None

    def process_initial_recipe(self, recipe_text):
        """Parse recipe structure with numbered step handling"""
        # Extract numbered steps first
        steps = re.split(r'\n\s*\d+\.\s', recipe_text)
        if len(steps) > 1:
            structured_recipe = {
                'steps': [s.strip() for s in steps[1:]]  # Skip first empty split
            }
            self.recipe_data = structured_recipe
            return self.recipe_data
        else:
            # Fallback to original processing
            self.recipe_data = process_recipe.invoke({"recipe_text": recipe_text})
            return self.recipe_data


    def generate_response(self, chef_message):
        """Generate trainee's response to chef's instruction"""
        # Ensure recipe_data is loaded before trying to access its length
        if self.recipe_data is None or 'steps' not in self.recipe_data:
            # Handle case where recipe data is not loaded or missing steps
            num_steps = 0
        else:
            num_steps = len(self.recipe_data['steps'])

        response = self.llm.invoke([HumanMessage(
            content=f"""You're a cooking trainee following instructions. Current step: {self.current_step+1}/{num_steps}.
            Last chef instruction: {chef_message}
            Respond with either:
            1. "next" to continue
            2. A clarification question about technique/ingredients
            3. "repeat" to hear the step again"""
        )])
        return response.content.strip().lower()

# Modified conversation interface
def automated_cooking_session(recipe_json):
    # Initialize agents
    trainee = TraineeAgent()
    # The recipe_json passed here should be the raw text of the recipe,
    # not a JSON string describing the recipe structure.
    # Let's assume for this fix that recipe_json is the raw text string
    # that the `process_recipe` tool can parse.
    # If the input is truly a JSON string as in the example, you'll need
    # to extract the 'instructions' field from the JSON first.

    # *** IMPORTANT ADJUSTMENT ***
    # The example recipe_json is a JSON string, but the process_recipe tool
    # expects raw recipe text. We need to extract the relevant part.
    # For this fix, let's assume the input 'recipe_json' parameter
    # in the automated_cooking_session function is the raw recipe text.
    # If you intend to pass a JSON string like the example, you need
    # to parse that JSON and extract the 'instructions' or full text.
    # For now, I will fix assuming the input parameter is the raw text
    # that the tool can process.
    # If your actual use case passes the JSON string, you need to add
    # JSON parsing here before calling trainee.process_initial_recipe.

    # Example of parsing the JSON string if needed:
    # import json
    # recipe_obj = json.loads(recipe_json)
    # raw_recipe_text = recipe_obj.get("instructions", "") # Or combine title, ingredients, instructions

    # Assuming recipe_json parameter is the raw text for now
    recipe_data = trainee.process_initial_recipe(recipe_json)

    # Add a check to ensure recipe_data and 'steps' exist before proceeding
    if not recipe_data or 'steps' not in recipe_data:
        print("Chef: Could not process the recipe. Please provide valid recipe text.")
        return

    print("Chef: Let's begin cooking Miso-Butter Roast Chicken!")

    for step_idx, step in enumerate(recipe_data['steps']):
        # Chef provides instruction
        # The input to the agent_executor for generating chef response
        # should probably be the current step instruction itself,
        # perhaps with some context from the conversation history.
        # Directly using the raw step text might be too simplistic.
        # Let's adjust the chef's input to be a request to explain the step.
        chef_response = agent_executor.invoke({
            "input": f"Explain step {step_idx+1}: {step}",
            "chat_history": memory.load_memory_variables({})["chat_history"] # Include memory
        })
        print(f"\nChef: {chef_response['output']}")

        # Trainee interaction loop
        while True:
            trainee_response = trainee.generate_response(chef_response['output'])

            if "next" in trainee_response:
                trainee.current_step = step_idx
                break

            elif "repeat" in trainee_response:
                print(f"Trainee: [Requesting repeat]")
                # No need to re-invoke the chef agent for repeat, just print the step again if desired,
                # or let the loop continue with the same chef_response message being used
                # by generate_response in the next iteration. The current setup re-uses it.
                pass # Continue the inner loop, trainee.generate_response gets the same chef_response

            else:  # Handle questions
                # The input to the agent_executor for handling trainee questions
                clarification = agent_executor.invoke({
                    "input": f"Trainee asks about the last instruction: {trainee_response}",
                    "chat_history": memory.load_memory_variables({})["chat_history"] # Include memory
                })
                print(f"Trainee: {trainee_response}")
                print(f"Chef: {clarification['output']}")

    print("\nChef: All steps completed! Enjoy your meal!")

# Example usage (adjusting the input to be raw text)
if __name__ == "__main__":
    # Provide the recipe as a raw text string here
    raw_recipe_text = """
Miso-Butter Roast Chicken With Acorn Squash Panzanella

Ingredients:
1 whole chicken (about 3-4 lbs)
2 tbsp white miso paste
4 tbsp unsalted butter, softened
1 tsp garlic powder
Salt and black pepper to taste
1 acorn squash, peeled and cubed
1 cup bread cubes (from stale bread)
2 tbsp olive oil
1 tbsp fresh sage, chopped

Instructions:
1. Preheat oven to 400°F (200°C).
2. In a small bowl, mix miso paste, softened butter, and garlic powder.
3. Pat the chicken dry. Rub the miso-butter mixture all over the chicken, under the skin, and inside the cavity. Season generously with salt and pepper.
4. Place the chicken in a roasting pan. Arrange acorn squash cubes and bread cubes around the chicken. Drizzle vegetables and bread with olive oil and sprinkle with chopped sage.
5. Roast for 60-75 minutes, or until the chicken is cooked through and the juices run clear (internal temperature of 165°F or 74°C).
6. Let the chicken rest for 10 minutes before carving. Serve with the roasted squash and panzanella.
"""

    automated_cooking_session(raw_recipe_text)

Chef: Let's begin cooking Miso-Butter Roast Chicken!


> Entering new AgentExecutor chain...

Invoking: `explain_cooking_technique` with `{'technique': 'preheat oven'}`


Detailed explanation of preheat oven...Preheating the oven is a crucial step in many baking and roasting recipes. It involves turning on the oven to the required temperature and allowing it to come up to that temperature before placing any food inside. This ensures that the oven environment is stable and hot enough to start cooking immediately, which helps achieve consistent results, especially for temperature-sensitive recipes. For instance, when set to 400°F (200°C), preheating usually takes about 10-15 minutes depending on the oven, but it's important to wait until the oven reaches the full temperature before cooking.

> Finished chain.

Chef: Preheating the oven is a crucial step in many baking and roasting recipes. It involves turning on the oven to the required temperature and allowing it to come up to that temp

KeyboardInterrupt: 

In [39]:
class TraineeAgent:
    def __init__(self, conversation_mode="mixed"):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)
        self.current_step = 0
        self.recipe_data = None
        self.conversation_mode = conversation_mode  # can be "substitution", "technique", "mixed", etc.

    def process_initial_recipe(self, recipe_text):
        """Parse recipe structure with numbered step handling"""
        # Extract numbered steps first
        steps = re.split(r'\n\s*\d+\.\s', recipe_text)
        if len(steps) > 1:
            structured_recipe = {
                'steps': [s.strip() for s in steps[1:]]  # Skip first empty split
            }
            self.recipe_data = structured_recipe
            return self.recipe_data
        else:
            # Fallback to original processing
            self.recipe_data = process_recipe.invoke({"recipe_text": recipe_text})
            return self.recipe_data

    def generate_response(self, chef_message):
        num_steps = len(self.recipe_data['steps']) if self.recipe_data and 'steps' in self.recipe_data else 0

        # Select prompt template based on mode
        if self.conversation_mode == "substitution":
            question_instruction = "Ask ONE SHORT question about possible ingredient substitutions for this step. If no substitution is relevant, say 'next'."
        elif self.conversation_mode == "technique":
            question_instruction = "Ask ONE SHORT question about the cooking technique or process in this step. If no question is relevant, say 'next'."
        elif self.conversation_mode == "mixed":
            question_instruction = "Ask ONE SHORT question about either ingredient substitutions or cooking technique for this step. If no question is relevant, say 'next'."
        else:
            question_instruction = "Ask ONE SHORT question about this step or say 'next'."

        prompt = f"""You're following a recipe with {num_steps} steps. Current step: {self.current_step+1}.
Last instruction: {chef_message}
{question_instruction}
"""

        response = self.llm.invoke([HumanMessage(content=prompt)])
        return response.content.strip().lower()

def automated_cooking_session(recipe_text):
    trainee = TraineeAgent()

    # Chef introduction
    print("Chef: 🧑🍳 Hello! I'm ChefAI, your virtual cooking assistant.")
    print("Chef: Please provide the recipe you'd like to cook today.\n")

    # Simulate trainee providing recipe (this is handled here, not in generate_response)
    print(f"Trainee: Here's the recipe I want to prepare:\n{'-'*30}")
    print(recipe_text)
    print(f"{'-'*30}\n")

    try:
        recipe_data = trainee.process_initial_recipe(recipe_text)
        if not recipe_data or not recipe_data.get('steps'):
            raise ValueError("No steps found in recipe")
    except Exception as e:
        print(f"Chef: Recipe processing failed - {str(e)}")
        return

    # Extract recipe title from first line
    recipe_title = recipe_text.splitlines()[0].strip()
    print(f"Chef: 🎉 Excellent choice! Let's cook {recipe_title} together!\n")

    # Rest of the cooking flow remains the same
    for step_idx, step in enumerate(recipe_data['steps']):
        print(f"\n{'='*30}\nStep {step_idx+1}/{len(recipe_data['steps'])}")

        # Include memory in the invoke call for context
        chef_response = agent_executor.invoke({
            "input": f"Present step {step_idx+1}: {step}",
            "chat_history": memory.load_memory_variables({})["chat_history"]
        })
        print(f"\nChef: {chef_response['output']}")

        attempts = 0
        while attempts < 3:
            # Now generate_response only handles responses after the initial recipe submission
            trainee_response = trainee.generate_response(chef_response['output'])
            print(f"\nTrainee: {trainee_response}")

            if "next" in trainee_response:
                trainee.current_step = step_idx # Update current step before moving
                print("\nChef: Moving to next step...")
                break

            elif "repeat" in trainee_response:
                print(f"\nChef: Repeating step {step_idx+1}:")
                print(f"Chef: {chef_response['output']}")
                attempts += 1

            else:
                # Include memory in the invoke call for context
                clarification = agent_executor.invoke({
                    "input": f"Trainee asks: {trainee_response}",
                     "chat_history": memory.load_memory_variables({})["chat_history"]
                })
                print(f"\nChef: {clarification['output']}")
                attempts += 1

    print("\nChef: 🍽️ Cooking complete! Enjoy your meal!\n")

# Usage with proper recipe format
if __name__ == "__main__":
    recipe = """Miso-Butter Roast Chicken With Acorn Squash Panzanella

    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions":
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½""-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside."""

    automated_cooking_session(recipe)

Chef: 🧑🍳 Hello! I'm ChefAI, your virtual cooking assistant.
Chef: Please provide the recipe you'd like to cook today.

Trainee: Here's the recipe I want to prepare:
------------------------------
Miso-Butter Roast Chicken With Acorn Squash Panzanella

    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flou